In [23]:
import pandas as pd
import csv
import numpy as np
from datetime import datetime

In [24]:
df1 = pd.read_csv("/Users/asteriag/Desktop/DATASCIENCEPROJECT_WORKFLOW/TimeStamp_Engineering/engineering_OF_segment1.csv",engine='python')

In [25]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 195564 entries, 0 to 195563
Data columns (total 56 columns):
 #   Column                             Non-Null Count   Dtype  
---  ------                             --------------   -----  
 0   CASE_ID                            195564 non-null  object 
 1   WRKFLW_TSK_MAP_GROUPED             195564 non-null  object 
 2   WRKFLW_TSK_MAP_DETAILED            195564 non-null  object 
 3   PATH_CRDATETIME                    195564 non-null  object 
 4   ITEM_CRUSER                        195564 non-null  object 
 5   WRKFLW_ORG_TYPE_CRUSER             180151 non-null  object 
 6   STATUS_ORDER                       195564 non-null  int64  
 7   LATEST_STATUS                      195564 non-null  object 
 8   WRKFLW_PROCESS                     195564 non-null  object 
 9   QUERY_PART                         195564 non-null  object 
 10  REFERRAL_ID                        98813 non-null   object 
 11  APPLICATION_ID                     1955

In [26]:
df1['WRKFLW_TSK_MAP_GROUPED'].nunique()


149

In [27]:
df1['WRKFLW_TSK_MAP_DETAILED'].nunique()

383

In [28]:
for i in range(len(df1)):
    id=df1.loc[i,'CASE_ID']
    status = df1.loc[i,"WRKFLW_TSK_MAP_GROUPED"]

In [29]:
ids = list(set(df1['CASE_ID'].values.tolist()))
ids

['715808|405750|629365',
 '972666|483471|741839',
 '1054093|520512|798128',
 '1066734|449459|690772',
 '1050537|504457|774178',
 '743459|364604|568533',
 '1064289|420025|650931',
 '1053150|404263|627184',
 '1142295|520751|798488',
 '1012540|350064|546236',
 '1052946|404680|627793',
 '1057230|410072|635933',
 '1151105|534349|817782',
 '825704|415589|644256',
 '1128332|513791|788068',
 '1120620|491103|754512',
 '1146762|528208|808944',
 '1095385|461746|709311',
 '1062267|456276|700712',
 '1046030|392563|611107',
 '1064458|420291|651344',
 '1009789|346698|541270',
 '904615|470079|721767',
 '825772|382232|595339',
 '1072480|431839|668735',
 '1073640|433328|670990',
 '1067405|473947|727270',
 '1020922|360245|561607',
 '1023484|412929|640119',
 '1064686|559523|856081',
 '1062442|417626|647339',
 '1055862|408147|737527',
 '1031137|404181|627078',
 '1156080|542022|829532',
 '860156|566481|864121',
 '1090042|454430|698263',
 '1059020|413115|640412',
 '1023665|363847|567319',
 '1024294|364847|56

In [30]:
S1S1 = pd.DataFrame()
S1S2 = pd.DataFrame()
S1S3 = pd.DataFrame()
S1S4 = pd.DataFrame()
S1S5 = pd.DataFrame()

#traverse each id
for id in ids:
    flag = True
    id_df = df1[df1['CASE_ID'] == id].loc[::-1].reset_index(drop = True)
    #id_df= id_df.iloc[::-1]
    sub_index,sendForAssessment_index,assessmentOutcomeNotS_index, assessmentOutcomeS_index, releaseOffer_index,acceptOrDefer_index = -1,-1,-1,-1,-1,-1
    #.loc[::-1].reset_index(drop = True) Reverse row order and reset index which let the first status stay at the first row:
    for index, row in id_df.iterrows():
        #record the index of first 'Submit' status per person
        if row['WRKFLW_TSK_MAP_GROUPED'] == 'Submit'and sub_index == -1:
            sub_index = index
        #record the index of first 'Send for Assessment' status per person
        elif row['WRKFLW_TSK_MAP_GROUPED'] == 'Send for Assessment'and sendForAssessment_index == -1:
            sendForAssessment_index = index
        #record the index of first 'Assessment Outcome S' status per person
        elif (row['WRKFLW_TSK_MAP_GROUPED'] == 'Assessment Outcome - Successful'and assessmentOutcomeS_index == -1):
            assessmentOutcomeS_index = index
        elif (row['WRKFLW_TSK_MAP_GROUPED'] == 'Assessment Outcome - Not Successful'and assessmentOutcomeNotS_index == -1):
            assessmentOutcomeNotS_index = index
        #record the index of first 'Release Offer' status per person
        elif row['WRKFLW_TSK_MAP_GROUPED'] == 'Release Offer' and releaseOffer_index == -1:
            releaseOffer_index = index
        #record the index of first 'Accept/Defer' status per person
        elif (row['WRKFLW_TSK_MAP_GROUPED'] == 'Accept' and acceptOrDefer_index == -1) or\
                (row['WRKFLW_TSK_MAP_GROUPED'] == 'Defer' and acceptOrDefer_index == -1):
            acceptOrDefer_index = index
    #save the person status between submit and sendforassessment into S1
    if sub_index != -1 and sendForAssessment_index != -1 and sub_index < sendForAssessment_index:
        if (assessmentOutcomeS_index != -1 and assessmentOutcomeS_index < sendForAssessment_index) or (assessmentOutcomeNotS_index != -1 and assessmentOutcomeNotS_index <sendForAssessment_index)\
                or (releaseOffer_index != -1 and releaseOffer_index <sendForAssessment_index) or (acceptOrDefer_index != -1 and acceptOrDefer_index < sendForAssessment_index):
            continue
        else:
            df_seg1 = id_df[sub_index:sendForAssessment_index+1]
            S1S1 = pd.concat([S1S1,df_seg1])
            flag = False
    #save the person status from submit and before sendforassessment into S1
    elif sub_index != -1 and sendForAssessment_index == -1 and assessmentOutcomeS_index == -1 and assessmentOutcomeNotS_index == -1\
        and releaseOffer_index == -1 and acceptOrDefer_index == -1:
        df_seg1 = id_df
        S1S1 = pd.concat([S1S1,df_seg1])
        flag = False
    #save the person status between sendforassessment and assessmentOutcome into S2
    if sub_index != -1 and sendForAssessment_index != -1 and assessmentOutcomeS_index != -1 and sub_index < sendForAssessment_index and sendForAssessment_index <assessmentOutcomeS_index:
        if (assessmentOutcomeNotS_index != -1 and assessmentOutcomeNotS_index <assessmentOutcomeS_index) or (releaseOffer_index != -1 and releaseOffer_index <assessmentOutcomeS_index) or (acceptOrDefer_index != -1 and acceptOrDefer_index < assessmentOutcomeS_index):
            continue
        else:
            df_seg2=id_df[sendForAssessment_index:assessmentOutcomeS_index+1]
            S1S2 = pd.concat([S1S2,df_seg2])
            flag = False
    elif sub_index != -1 and sendForAssessment_index != -1 and assessmentOutcomeNotS_index != -1 and sub_index < sendForAssessment_index and sendForAssessment_index <assessmentOutcomeNotS_index:
        if (assessmentOutcomeS_index != -1 and assessmentOutcomeS_index < assessmentOutcomeNotS_index) or (releaseOffer_index != -1 and releaseOffer_index <assessmentOutcomeNotS_index) or (acceptOrDefer_index != -1 and acceptOrDefer_index < assessmentOutcomeNotS_index):
            continue
        else:
            df_seg2=id_df[sendForAssessment_index:assessmentOutcomeNotS_index+1]
            S1S2 = pd.concat([S1S2,df_seg2])
            flag = False
    #save the person status from sendforassessment and before assessmentOutcome into S2
    elif sub_index != -1 and sendForAssessment_index != -1 and assessmentOutcomeS_index == -1 and assessmentOutcomeNotS_index == -1\
        and releaseOffer_index == -1 and acceptOrDefer_index == -1 and sub_index < sendForAssessment_index:
        df_seg2=id_df[sendForAssessment_index:]
        S1S2 = pd.concat([S1S2,df_seg2])
        flag = False
    #save the person status between assessmentOutcome and relaseoffer into S3
    if sub_index != -1 and sendForAssessment_index != -1 and assessmentOutcomeS_index != -1 and releaseOffer_index != -1 and sub_index < sendForAssessment_index and sendForAssessment_index < assessmentOutcomeS_index and assessmentOutcomeS_index < releaseOffer_index:
        if (assessmentOutcomeNotS_index != -1 and assessmentOutcomeNotS_index <releaseOffer_index) or (acceptOrDefer_index != -1 and acceptOrDefer_index <releaseOffer_index):
            continue
        else:
            df_seg3=id_df[assessmentOutcomeS_index:releaseOffer_index +1]
            S1S3 = pd.concat([S1S3,df_seg3])
            flag = False
    #save the person status from assessmentOutcome and before relaseoffer into S3
    elif sub_index != -1 and sendForAssessment_index != -1 and assessmentOutcomeS_index != -1 and assessmentOutcomeNotS_index == -1 and releaseOffer_index == -1\
            and acceptOrDefer_index == -1 and sub_index < sendForAssessment_index and sendForAssessment_index <assessmentOutcomeS_index:
        df_seg3=id_df[assessmentOutcomeS_index:]
        S1S3 = pd.concat([S1S3,df_seg3])
        flag = False
    #save the person status between relaseoffer and acceptordefer into S4
    if sub_index != -1 and sendForAssessment_index != -1 and assessmentOutcomeS_index != -1 and releaseOffer_index!= -1 and acceptOrDefer_index != -1 and sub_index < sendForAssessment_index and sendForAssessment_index < assessmentOutcomeS_index and assessmentOutcomeS_index < releaseOffer_index and releaseOffer_index <acceptOrDefer_index:
        if assessmentOutcomeNotS_index != -1 and assessmentOutcomeNotS_index < acceptOrDefer_index:
            continue
        else:
            df_seg4=id_df[releaseOffer_index:acceptOrDefer_index+1]
            S1S4 = pd.concat([S1S4,df_seg4])
            flag = False
    #save the person status from relaseoffer and before acceptordefer into S4
    elif sub_index != -1 and sendForAssessment_index != -1 and assessmentOutcomeS_index != -1 and releaseOffer_index != -1 and acceptOrDefer_index == -1 and sub_index < sendForAssessment_index and sendForAssessment_index < assessmentOutcomeS_index and assessmentOutcomeS_index < releaseOffer_index and assessmentOutcomeNotS_index == -1:
        df_seg4=id_df[releaseOffer_index:]
        S1S4 = pd.concat([S1S4,df_seg4])
        flag = False
    #save the person who didn't belong to before four segements
    if flag == True:
        S1S5 = pd.concat([S1S5,id_df])

In [31]:
header = ["CASE_ID","WRKFLW_TSK_MAP_GROUPED","WRKFLW_TSK_MAP_DETAILED","PATH_CRDATETIME"]


In [32]:
df1['CASE_ID'].nunique()

12957

In [33]:
S1S1['CASE_ID'].nunique()

11881

In [34]:
S1S2['CASE_ID'].nunique()

10262

In [35]:
S1S3['CASE_ID'].nunique()

6322

In [36]:
S1S4['CASE_ID'].nunique()

6265

In [37]:
S1S5['CASE_ID'].nunique()

179

In [38]:
#segment1
S1S1.head(n=10)
S1S1.to_csv("engineering_OF_segment1_workflowS1.csv",index=False,columns=header)

In [39]:
#segmnent2
S1S2.head(n=10)
S1S2.to_csv("engineering_OF_segment1_workflowS2.csv",index=False,columns=header)

In [40]:
#segment3
S1S3.head(n=10)
S1S3.to_csv("engineering_OF_segment1_workflowS3.csv",index=False,columns=header)

In [41]:
#segment4
S1S4.head(n=10)
S1S4.to_csv("engineering_OF_segment1_workflowS4.csv",index=False,columns=header)

In [42]:
#appliers didn't count into those 4 segments
S1S5.head(n=10)
S1S5.to_csv("engineering_OF_segment1_workflowS5.csv",index=False,columns=header)